# CLIPtionary Attack Notebook
<h2>License</h2>
Copyright (c) 2021 Bearsharktopusdev

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# HOW TO USE
<h2>Upload your image in the file thing. Put the image path in Parameters as indicated. Fill in other parameters as desired. Restart + Run All and then go get some coffee because it'll take a little bit.</h2>

# Imports & Shit

In [ ]:
#@markdown
!nvidia-smi

In [ ]:
#@markdown
%cd /content/

!git clone https://github.com/openai/CLIP.git

%cd /content/CLIP/

!pip install ftfy
!pip install tqdm
from tqdm import trange

import imageio
import torchvision

from IPython import display
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import clip
!pip install kornia
import kornia
import torch
import torch.nn.functional as F

from PIL import Image

import random

clip.available_models()

#RN50x4
#ViT-B/32

import numpy as np

# Load the model
perceptor, preprocess = clip.load('ViT-B/32', jit=True) # usually use vit
perceptor = perceptor.eval()

clear_output()

In [ ]:
#@markdown

%cd /content/
!git clone https://github.com/first20hours/google-10000-english

clear_output()

In [ ]:
#@markdown
file = open("/content/google-10000-english/google-10000-english-usa-no-swears-short.txt", "r")
short_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-medium.txt", "r")
medium_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-long.txt", "r")
long_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/20k.txt", "r")
full_list = file.read().splitlines()
file.close()

# Parameters

In [ ]:
check_amount = 100 #100, number of terms to contest per epoch
epochs = 10 #10, number of iterations of the checking algorithm for the initial seeding
culling_threshold = 1.2 #1.2, any phrase worth less than this many words is culled from the list. 
# for example, with a threshold of 1 and 100 words, any word that scores less than 0.01 (1/100) is culled
# Recommended to be kept between 1 and 2

seed_words = "giant, tall, grotesque, sirenhead, daytime, thin, slender, slimyswampghost, photo" #"", comma separated list of words to seed the initial epochs with.
image = "/content/sirenhead.jpg" #don't delete /content/, just upload to google colab and put your filename there

dictionary_from_top = 5 #Adds this many words from the top of the frequency-rated dictionary to the dictionary attack
dictionary_from_random = 5 #Adds this many words randomly from the frequency-rated dictionary to the dictionary attack

iterative_guess_words = "sirenhead, photograph of, slimyswampghost" #"", comma separated list of words for the iterative guess & check step

# Definitions

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

image = preprocess(Image.open(image)).unsqueeze(0).to(device)
image_features = model.encode_image(image)

winner_list = []

def rate_words(input_list):
  print("rating")
  text = clip.tokenize(input_list).to(device)

  with torch.no_grad():
      text_features = model.encode_text(text)
      
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  probs_list = probs.tolist()[0]
  input_probs_zip = zip(input_list, probs_list)
  input_probs_list = []
  for i in input_probs_zip:
    input_probs_list.append(i)
  result = [i for i in input_probs_list if i[1] >= max(culling_threshold/len(input_list), 0.5/check_amount)]
  result = remove_duplicates(result)
  result.sort(key=lambda x:x[1], reverse=True)

  return result

def refill_list(input_list):
  random_list = random.sample(full_list, check_amount)
  while len(input_list) < check_amount:
    input_list.append(random_list[0])
    random_list.pop(0)

def remove_duplicates(input_list):
    return list(set([i for i in input_list]))

def attack(input_list):
  for k in input_list:
    global winner_list
    winner_list = remove_duplicates(winner_list)
    thing = [x[0] for x in winner_list]
    thinglen = len(thing)
    thinglenlen = len(str.split(thing[0]))+1
    new_list = [thing]*thinglenlen
    new_new_list = []
    for i in range(thinglenlen):
      for j in new_list[i]:
        newj = str.split(j)
        newj.insert(i, k)
        newj = " ".join(newj)
        new_new_list.append(newj)
    for i in thing:
      new_new_list.append(i)
    new_new_list.append(k)

    winner_list = rate_words(new_new_list)

    print('\n---')
    winner_list.sort(key=lambda x:x[1], reverse=True)
    for i in winner_list:
      print(i)

# Dictionary Seeding

In [ ]:
seed_word_list = seed_words.split(", ")
initial_attack = full_list[0:check_amount - len(seed_word_list)]
initial_attack = initial_attack + seed_word_list

for i in range(epochs):
  refill_list(initial_attack)
  winner_list = rate_words(initial_attack)
  initial_attack = [x[0] for x in winner_list]

  print('\n---')
  for i in winner_list:
    print(i)

# Self-Comparing Attack

In [ ]:
self_comp_attack = []

new = [x[0] for x in winner_list]
for k in new:
  for i in new:
    self_comp_attack.append(k + " " + i)
  self_comp_attack.append(k)

winner_list = rate_words(self_comp_attack)
self_comp_attack = [x[0] for x in winner_list]

print('\n---')
for i in winner_list:
  print(i)

# Iterative Dictionary Attack

In [ ]:
dictionary_attack = full_list[0:dictionary_from_top]
random_list = random.sample(full_list, dictionary_from_random)
dictionary_attack = dictionary_attack + random_list

attack(dictionary_attack)

# Synonym/Antonym Attack

# iterative Guess & Check

In [ ]:
words_list = iterative_guess_words.split(", ")

attack(words_list)